In [0]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [0]:
#import main model & test by the testing sample - '/gdrive/My Drive/18-winter/정리판/test_file/'
import tensorflow as tf
import numpy as np

#path to the directory
dirname = '/gdrive/My Drive/18-winter/정리판/'

#saved model
model = dirname + 'main/model_2/'

tf.reset_default_graph()

with tf.Session() as sess:
  saver = tf.train.import_meta_graph(model + 'main_model.meta')
  saver.restore(sess, tf.train.latest_checkpoint(model))
  
  graph = tf.get_default_graph()
  
  X = graph.get_tensor_by_name("X:0")
  Y = graph.get_tensor_by_name("Y:0")
  
  #WEIGHTS
  W1 = graph.get_tensor_by_name("w1:0")
  W2 = graph.get_tensor_by_name("w2:0")
  W3 = graph.get_tensor_by_name("w3:0")
  W = graph.get_tensor_by_name("w:0")

  #BIASES
  b1 = graph.get_tensor_by_name("b1:0")
  b2 = graph.get_tensor_by_name("b2:0")
  b3 = graph.get_tensor_by_name("b3:0")
  b = graph.get_tensor_by_name("b:0")
  
  #layers
  layer1 = tf.nn.sigmoid(tf.matmul(X, W1) + b1)
  layer2 = tf.nn.relu(tf.matmul(layer1, W2) + b2)
  layer3 = tf.nn.relu(tf.matmul(layer2, W3) + b3)
  
  #dropout
  keep_prob = tf.placeholder(tf.float32, name = "keep_prob")
  dropout = tf.nn.dropout(layer3, keep_prob)
  
  y = tf.nn.softmax(tf.matmul(dropout, W) + b)#추측값
    
  e = [0,0,0,0,0,0,0,0,0]#각 class 마다 일치한 test sample 개수
  T = 0#ans와 일치한 test sample 개수
  m = 0#전체 test sample 개수
  classes = ['뉴스','리모컨','소리작게','소리크게','시간','오늘날씨','오늘일정','지니야','클로바']
  
  for d in range(9):
    filename = classes[d]
    
    for count in range(30):
      data = np.load(dirname + 'test_file/' + filename + '/%d.npz'%count)
      
      y_hat = sess.run(y, feed_dict = {X: data['X'], keep_prob: 1.0})
      
      length = len(y_hat)
      ans_list = np.zeros(shape = [9], dtype = np.float32)#추정치
      
      for num in range(length):
        ans_list = ans_list + y_hat[num]
        
      ans_list /= num
      ans = np.argmax(ans_list, 0)#추정치에서 가장 큰 값을 가진 칸의 index
      
      m = m + 1
      if d == ans:
        T = T + 1
        e[d] = e[d] + 1
      
      #print(d, ans)
      #print(ans_list)
      
  for l in range(9):
    print(e[l])

  test_accuracy = T/m
  print("test_accuracy: %f"%test_accuracy)

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /gdrive/My Drive/18-winter/정리판/main/model_2/main_model
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
18
1
0
30
25
21
26
23
29
test_accuracy: 0.640741
